In [1]:
import numpy as np
import pandas as pd
import pickle
import plotly.graph_objects as go
from copy import copy, deepcopy
from pprint import pprint
from Geometry3D import Point, distance, HalfLine

from rich.console import Console
rprint = Console(style="on #272727").print

#suppress scientific notation
np.set_printoptions(suppress=True, precision=8)

In [10]:
import threader, geometry_helpers, gcode, gcline, tlayer, util, cura4layer
import parsers.cura4
from parsers import cura4
from threader import TLayer, Threader, GCodeException
from util import find
from danutil import dupcount
#page_wide()

#Attempt to reload modules automatically
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#Fusion: smaller complex shape v1
gcode_file = '/Users/dan/r/thread_printer/stl/test2/Smaller complex shape - cura.gcode'
cura_project = '/Users/dan/r/thread_printer/stl/test2/Smaller complex shape - cura.3mf'

In [ ]:
thread_transform = cura4.parse_3mf(cura_project)
thread_transform

In [4]:
# From cura4.parse_3mf(cura_project) so I don't have to run that every time
thread_transform = np.array([39.26039839, 46.27034378, 23.87050056])

#See thread_from_fusion.py to get thread path
tpath = [
		([33.7300, 17.1077, 11.4653], [56.0477, 43.4545, 11.4653]),
		([56.0477, 43.4545, 11.4653], [53.2410, 59.2656, 11.4653]),
		([53.2410, 59.2656, 11.4653], [26.9901, 69.3288, 42.0560]),
		([26.9901, 69.3288, 42.0560], [26.9901, 69.3288, 47.7410]),
]
tpath = np.array(tpath)

#Do it this way so I can add transform if needed
#tpath = np.insert(tpath, 0, [[0,0,0], tpath[0,0]], axis=0)

#Temporary fix for TODO in cura4.parse3mf
thread_transform[2] = 0

g = gcode.GcodeFile(gcode_file, layer_class=TLayer)
t = Threader(g)

#start_layer = next(layer for layer in g.layers if (layer.z + layer.layer_height) >= tpath[0][1][2])
start_layer = g.layers[56]

thread_geom = list([geometry_helpers.GSegment(Point(*s), Point(*e)) for s,e in tpath])

In [7]:
stepsobj = t.route_layer(thread_geom, start_layer)

Route 4-segment thread through layer:
  <Layer 56 at Z=11.4; corners: ((15.235, 16.791), (63.186, 75.723)); 376 lines>

        0. <{ 33.73,  17.11,  11.47}←→{ 56.05,  43.45,  11.47}>

        1. <{ 56.05,  43.45,  11.47}←→{ 53.24,  59.27,  11.47}>

        2. <{ 53.24,  59.27,  11.47}←→{ 26.99,  69.33,  42.06}>

        3. <{ 26.99,  69.33,  42.06}←→{ 26.99,  69.33,  47.74}>

...

5/5 unprinted segments that intersect current thread segment

2: Nothing not intersecting thread to print

<Step 7 (0 segments)>: Print 5 overlapping layers segments
  Printer(Bed(0, 0, ⚓︎{  0.00,   0.00,   0.00}), Ring(86.12°, { 36.00,  28.00,   0.00}))

Adding 5/5 unprinted gcsegs to Step

Done with thread for this layer; 0 gcode lines left

In [8]:
allgc = t.steps.gcode()

————— START STEP 0: Move thread to avoid 280/298 segments of non-intersecting geometry —————

————— START STEP 1: Print non-intersecting geometry —————

new line from 21768

new line from 21772

new line from 21797

...

————— START STEP 7: Print 5 overlapping layers segments —————

new line from 21767

new line from 21771

new line from 21815

  new step line: G0 X61.032 Y57.893 ; [] ; ---- fake inter-step move from 21767

In [11]:
dupcount([l.lineno for l in allgc if isinstance(l.lineno, int)])

{}

In [ ]:
len(set([l.lineno for l in start_layer.lines])), len(allgc), len([l for l in allgc if not l.fake])

In [ ]:
[l for l in allgc if not l.fake and l.lineno not in start_layer.lines]

In [23]:
with open('/tmp/steps.gcode', 'w') as f:
    f.write('\n'.join(util.construct_lines_rel2abs(allgc)[0]))

G1 E-21.505 F8000 ; [] ; Ring move from 0.00° to 21.50°
G1 E-25.079 F8000 ; [] ; Ring move from 21.50° to 46.58°
G1 E-39.538 F8000 ; [] ; Ring move from 46.58° to 86.12°


In [ ]:
### Problem: there are repeated line numbers, such as 21821

In [ ]:
stepsobj.steps[1].gcsegs

In [ ]:
_.printed

In [ ]:
stepsobj.steps[5].gcode()[-1]

In [ ]:
stepsobj.steps[6].gcode()

In [ ]:
find(stepsobj.steps[5].gcode(), lambda l:l.is_xymove(), rev=True)

In [ ]:
stepsobj.plot()

In [ ]:
segs = stepsobj.steps[1].gcsegs
pairs = [(s1,s2) for (s1,s2) in zip(segs[:-1], segs[1:]) if s2.gc_lines.first.lineno - s1.gc_lines.last.lineno > 2]
pairs

In [ ]:
s = [seg for step in stepsobj.steps for seg in step.gcsegs if 21880 in seg.gc_lines]
s

In [ ]:
print(s[0], '\n', s[0].gc_lines)

In [ ]:
segs.sort(key=lambda s:s.gc_lines.first.lineno)
for seg1, seg2 in zip(segs[:-1], segs[1:]):
    s1l = seg1.gc_lines.last.lineno
    s2f = seg2.gc_lines.first.lineno
    if s1l < 21800 or s2f > 21900:
        continue
    print((seg1.gc_lines.first.lineno, seg1.gc_lines.last.lineno),
          (seg2.gc_lines.first.lineno, seg2.gc_lines.last.lineno),
         ' = ', s2f-s1l)

In [ ]:
gcl = gcline.GCLines(g.lines)

In [ ]:
gcl[21875:21880]

In [ ]:
gcline.GCLine(comment='hello').lineno

In [ ]:
import rich
rich.terminal_theme.DEFAULT_TERMINAL_THEME = rich.terminal_theme.MONOKAI

In [ ]:
fig = go.Figure()
t.printer.bed.plot(fig)
start_layer.plot(fig, only_outline=False)
fig.add_trace(go.Scatter(**geometry_helpers.segs_xy(
	#*t.steps.flat_thread,
	*thread_geom,
	**threader.Step.style['thread'])))
t.printer.ring.plot(fig)

fig.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=0, pad=0),
		showlegend=False,)
fig.show()

In [ ]:
###### seg0 = list(start_layer.model_isecs.keys())[3]
start_layer.model_isecs[seg0]

In [ ]:
stepsobj.steps[1].printer.thread_seg

In [ ]:
id(stepsobj.steps[0].printer)

In [ ]:
id(stepsobj.steps[0].printer)

In [ ]:
ring0 = stepsobj.steps[0].printer.ring
ring1 = copy(ring0)
ring0, ring1

In [ ]:
ring1.set_angle(0)
ring0, ring1

In [ ]:
id(ring0), id(ring1)

In [ ]:
fig = go.Figure()
stepsobj49.steps[5].plot_gcsegments(fig)
fig.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=40, pad=0),
		showlegend=False,)

fig.show('notebook')

In [ ]:
stepsobj49.steps[5].gcsegs[-1].gc_lines[14922]

In [ ]:
g.lines[14920:14925]

In [ ]:
stepsobj49.steps[2].printer.ring.initial_angle, stepsobj49.steps[2].printer.ring.angle

In [ ]:
gc = stepsobj49.gcode()
with open('/tmp/l49.gcode', 'w') as f:
	f.write(gc.construct())

In [ ]:
g.layers[49]

In [ ]:
ogc = g.layers[49].lines
rprint(ogc.summary())
sorted(ogc[set(gc._index.keys()).difference(ogc._index.keys())])

In [ ]:
stepsobj49.steps[1].gcsegs[-1].gc_lines

In [ ]:
stepsobj50 = t.route_layer(thread_geom, g.layers[50])
stepsobj50.plot(g.layers[49])

In [ ]:
stepsobj60 = t.route_layer(thread_geom, g.layers[60])
stepsobj60.plot()

In [ ]:
fig = go.Figure()
layer = g.layers[49]
layer.plot(fig, ['green'], ['green'], only_outline=False)

thread = layer.flatten_thread(thread_geom)
layer.intersect_model(thread)

fig.add_trace(go.Scatter(**geometry_helpers.segs_xy(*thread,
	marker_size=8,
	line=dict(color='red', dash='dot', width=4))))

newthread = layer.anchor_snap(thread)

fig.add_trace(go.Scatter(**geometry_helpers.segs_xy(*newthread,
	marker=dict(color='lightblue', size=8),
	line=dict(color='yellow', dash='dot', width=1))))

fig.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=40, pad=0),
		showlegend=False,
)

fig.show('notebook')

In [ ]:
for seg, color in zip(thread_geom, plotly.colors.qualitative.Set2):
	fig.add_trace(go.Scatter3d(**geometry_helpers.segs_xyz(seg, mode='lines',
		line=dict(color=color, dash='dot', width=4))))

fig.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=40, pad=0),
		showlegend=False,
)
fig.update_scenes(
		camera=dict(
			projection={'type': 'orthographic'},
			# eye={'x': 0, 'y': 1, 'z': 0},
		),
)

fig.show('notebook')

In [ ]:
fig = go.Figure()
layer = g.layers[49]
layer.plot(fig, ['green'], ['green'], only_outline=False)

thread = layer.flatten_thread(thread_geom)
layer.intersect_model(thread)

fig.add_trace(go.Scatter(**geometry_helpers.segs_xy(*thread,
	marker_size=8,
	line=dict(color='red', dash='dot', width=4))))

newthread = layer.anchor_snap(thread)

fig.add_trace(go.Scatter(**geometry_helpers.segs_xy(*newthread,
	marker=dict(color='lightblue', size=8),
	line=dict(color='yellow', dash='dot', width=1))))

fig.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=40, pad=0),
		showlegend=False,
)

fig.show('notebook')

In [ ]:
import zipfile
import xml.etree.ElementTree as ET
import numpy as np

namespace = {
		"3mf": "http://schemas.microsoft.com/3dmanufacturing/core/2015/02",
		"m"  : "http://schemas.microsoft.com/3dmanufacturing/material/2015/02"
}

filename = '/Users/dan/data/projects/research/thread_printer/stl/test2/Smaller complex shape v1 - cura.3mf'
archive = zipfile.ZipFile(filename, "r")
mfroot = ET.parse(archive.open("3D/3dmodel.model"))

In [ ]:
mins = [float( 'inf')]*3
maxs = [float('-inf')]*3
for vertex in mfroot.findall("./3mf:resources/3mf:object/3mf:mesh/3mf:vertices/3mf:vertex", namespace):
	vals = list(map(float, [vertex.get(a) for a in 'xyz']))
	mins = [min(old, new) for old, new in zip(mins, vals)]
	maxs = [max(old, new) for old, new in zip(maxs, vals)]
mins, maxs

In [ ]:
import zipfile
import xml.etree.ElementTree as ET
import numpy as np

namespace = {
		"3mf": "http://schemas.microsoft.com/3dmanufacturing/core/2015/02",
		"m"  : "http://schemas.microsoft.com/3dmanufacturing/material/2015/02"
}

filename = '/Users/dan/data/projects/research/thread_printer/stl/test2/Smaller complex shape v1 - cura.3mf'
archive = zipfile.ZipFile(filename, "r")
mfroot = ET.parse(archive.open("3D/3dmodel.model"))

In [ ]:
mins = [float( 'inf')]*3
maxs = [float('-inf')]*3
for vertex in mfroot.findall("./3mf:resources/3mf:object/3mf:mesh/3mf:vertices/3mf:vertex", namespace):
	vals = list(map(float, [vertex.get(a) for a in 'xyz']))
	mins = [min(old, new) for old, new in zip(mins, vals)]
	maxs = [max(old, new) for old, new in zip(maxs, vals)]
mins, maxs

In [ ]:
np.vstack([threadstart, threadstart3mf, gthreadstart, tp[0]])

In [ ]:
verts3mf = np.unique(objects[2][::500,:2], axis=0)
tp_cent_trans = tp_cent + obj_bb[2]['tr'] + obj_bb[3]['tr'] - [0, 5.75, 0]

fig_t = go.Figure()

#GCode
start_layer.plot(fig_t, only_outline=False)

#Translated object from 3MF
trans_3mf = (objects[1] + obj_bb[3]['tr'])[::10,:2]
fig_t.add_trace(go.Scatter(name='trans_3mf', x=trans_3mf[:,0], y=trans_3mf[:,1], mode='markers'))

# fig_t.add_trace(go.Scatter(name='tp', x=tp[:,0], y=tp[:,1]))
# fig_t.add_trace(go.Scatter(name='tp_cent', x=tp_cent[:,0], y=tp_cent[:,1]))
# fig_t.add_trace(go.Scatter(name='verts3mf', x=verts3mf[:,0], y=verts3mf[:,1], mode='markers'))
fig_t.add_trace(go.Scatter(name='tp_cent_trans', x=tp_cent_trans[:,0], y=tp_cent_trans[:,1]))


fig_t.update_layout(template='plotly_dark',# autosize=False,
		yaxis={'scaleanchor':'x', 'scaleratio':1, 'constrain':'domain'},
		margin=dict(l=0, r=20, b=0, t=0, pad=0),
		showlegend=False,)
fig_t.show('notebook')